In [282]:
import os
import shutil 
import csv
import numpy as np
#import numpy as np
#import pandas as pd
#from sklearn.preprocessing import StandardScaler
from utils import *

In [283]:
band_name = 'v8'  
window_size = 6 

In [284]:
path = 'sources/Ini_sources/'
#print(os.getcwd())
data_1 = make_sequences_y1(band_name, path, band_name)
print(len(data_1))

10


In [285]:
''' Writing out initial sequences
file_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')
for line in data_1:
    file_ini.write(str(line) +'\n')
    
file_ini.close()
'''

" Writing out initial sequences\nfile_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')\nfor line in data_1:\n    file_ini.write(str(line) +'\n')\n    \nfile_ini.close()\n"

In [286]:
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().rounding = ROUND_HALF_UP
l = len(data_1)
h = int(round(Decimal(l)/2,0))
i = int(round(Decimal(h)/2,0))
indexes = [i, l-(h-i)]
print(indexes)

[3, 8]


In [287]:
train = data_1[:indexes[0]] + data_1[indexes[1]:]
test  = data_1[indexes[0]:indexes[1]]
print(len(train))

5


In [71]:
data_test_1 = get_x(test, window_size=window_size)

file_csv = open('sources/test_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y'])
for i in range(data_test_1.shape[0]):
    filewriter.writerow(data_test_1[i,:])
file_csv.close()

# Creating additional samples for train set

In [335]:
data = get_x(train, window_size=window_size).tolist()
print(len(data))
for line in data:
    print(line)

61
[3.6325600000000122, 3.630423999999948, 3.626299999999901, 3.6202769999999873, 3.613411000000042, 3.605570999999941, 1.0]
[3.630423999999948, 3.626299999999901, 3.6202769999999873, 3.613411000000042, 3.605570999999941, 3.598645000000033, 1.0]
[3.626299999999901, 3.6202769999999873, 3.613411000000042, 3.605570999999941, 3.598645000000033, 3.5925459999999703, 1.0]
[3.6202769999999873, 3.613411000000042, 3.605570999999941, 3.598645000000033, 3.5925459999999703, 3.5861169999999447, 1.0]
[3.613411000000042, 3.605570999999941, 3.598645000000033, 3.5925459999999703, 3.5861169999999447, 3.5819499999998925, 1.0]
[3.605570999999941, 3.598645000000033, 3.5925459999999703, 3.5861169999999447, 3.5819499999998925, 3.5790140000000292, 1.0]
[3.598645000000033, 3.5925459999999703, 3.5861169999999447, 3.5819499999998925, 3.5790140000000292, 3.5772160000000213, 1.0]
[3.5925459999999703, 3.5861169999999447, 3.5819499999998925, 3.5790140000000292, 3.5772160000000213, 3.5766069999999672, 1.0]
[3.58611699

# Negative sequence

In [336]:
for line in data:
    if abs(line[0] - 3.5738) <= 0.0001:
            print(line)
            print(create_deltas(line, y_label=True))

[3.573853999999983, 3.574555000000032, 3.578021999999919, 3.578895999999986, 3.5815470000000005, 3.586221000000023, 1.0]
[0, -0.0007010000000491345, -0.003466999999886866, -0.0008740000000670989, -0.002651000000014392, -0.0046740000000227155]


In [337]:
data_add = []
for el in data:
    #print('1', el)
    seq = []
    for cell in el[:-1]:
        seq.append(-cell)
    seq.append(1.0)
    #print('2', seq)
    data_add.append(seq)

for el in data_add:
    data.append(el)
print(len(data))

122


In [338]:
data_add = []
for el in data:
    #print('1', el)
    el = el[:-1]
    el.reverse()
    el.append(1.)
    #print('2', el)
    data_add.append(el)

for el in data_add:
    data.append(el)
print(len(data))

244


In [339]:
def create_deltas(line, y_label=False):
    if y_label == True:
        line = line[:-1]
    deltas = [0]
    for i in range(len(line)-1):
        delta = line[i] - line[i+1]
        deltas.append(delta)
    return deltas

def create_shift_in_deltas(data, shift):
    data_add = []

    for line in data:
        #print('1', line)
        #print(shift)
        deltas = create_deltas(line, y_label=True)
        min_delta = min(deltas[1:])
        #print(min_delta, shift)
        if abs(shift) > abs(min_delta):
            continue
        #print('d', deltas)
        new = [line[0]]
        ini = line[0]
        for i in range(1, len(line[:-1])):
            ini = ini - deltas[i] - shift
            new.append(ini)
        new.append(1.)
        if abs(line[0] - 3.5738) <= 0.0001:
            print(new)
        #print(new)
            print('1', deltas)
            print('2',create_deltas(new, y_label=True))
        data_add.append(new)
    
    return data_add

In [340]:
shifts = [i for i in np.arange(-0.015, 0.015, 0.0005) if abs(i) > 0.00001]
data_add_res = []
for shift in shifts:
    data_add = create_shift_in_deltas(data, shift)
    data_add_res += data_add
print(len(data_add_res))

#print(data_add_res[2715:2720])

[3.573853999999983, 3.5790550000000323, 3.5870219999999193, 3.5923959999999866, 3.599547000000001, 3.608721000000024, 1.0]
1 [0, -0.0007010000000491345, -0.003466999999886866, -0.0008740000000670989, -0.002651000000014392, -0.0046740000000227155]
2 [0, -0.005201000000049305, -0.007966999999887037, -0.005374000000067269, -0.007151000000014562, -0.009174000000022886]
[3.573853999999983, 3.578555000000032, 3.586021999999919, 3.590895999999986, 3.5975470000000005, 3.606221000000023, 1.0]
1 [0, -0.0007010000000491345, -0.003466999999886866, -0.0008740000000670989, -0.002651000000014392, -0.0046740000000227155]
2 [0, -0.004701000000049138, -0.00746699999988687, -0.0048740000000671024, -0.006651000000014395, -0.008674000000022719]
[3.573853999999983, 3.578055000000032, 3.5850219999999187, 3.5893959999999856, 3.595547, 3.6037210000000224, 1.0]
1 [0, -0.0007010000000491345, -0.003466999999886866, -0.0008740000000670989, -0.002651000000014392, -0.0046740000000227155]
2 [0, -0.004201000000048971,

In [341]:
'''data_add = []
for line in data:
    deltas = create_deltas(line, y_label=True)
    #print(line)
    #print('d ', deltas)
    if all(delta <= 0.1 for delta in deltas) is True:
        cell_max = m
        ax(map(abs,deltas))
        cell_min = min(map(abs,deltas[1:]))
        while (cell_max < 0.05):
            deltas = [d*10 for d in deltas]
            cell_max *= 10
            new = []
            ini = line[0]
            for i in range(len(deltas)):
                ini = ini - deltas[i]
                new.append(ini)
            new.append(1.)
            if abs(line[0] - 3.5738) <= 0.0001:
                print(line)
                print(create_deltas(line, y_label=True))
                print('1', new)
    
            #print('n ',new)
            data_add.append(new)
            
            if cell_min >= 0.002:
                deltas2 = [d/2 for d in deltas]
                new = []
                #print('1', deltas)
                #print('2',deltas2)
                ini = line[0]
                for i in range(len(deltas2)):
                    ini = ini - deltas2[i]
                    new.append(ini)
                new.append(1.)
                if abs(line[0] - 3.5738) <= 0.0001:
                    print('2',new)
                #print('n ',new)
                data_add.append(new)
            
print(len(data_add))
'''

"data_add = []\nfor line in data:\n    deltas = create_deltas(line, y_label=True)\n    #print(line)\n    #print('d ', deltas)\n    if all(delta <= 0.1 for delta in deltas) is True:\n        cell_max = m\n        ax(map(abs,deltas))\n        cell_min = min(map(abs,deltas[1:]))\n        while (cell_max < 0.05):\n            deltas = [d*10 for d in deltas]\n            cell_max *= 10\n            new = []\n            ini = line[0]\n            for i in range(len(deltas)):\n                ini = ini - deltas[i]\n                new.append(ini)\n            new.append(1.)\n            if abs(line[0] - 3.5738) <= 0.0001:\n                print(line)\n                print(create_deltas(line, y_label=True))\n                print('1', new)\n    \n            #print('n ',new)\n            data_add.append(new)\n            \n            if cell_min >= 0.002:\n                deltas2 = [d/2 for d in deltas]\n                new = []\n                #print('1', deltas)\n                #print('

In [342]:
for el in data_add_res:
    data.append(el)

for el in data_add:
    data.append(el)
print(len(data))   

9926


In [25]:
'''
data_add = []
for el in data:
    multiplicate = [2,2.5,3,0.5,0.2,0.1]
    for value in multiplicate:
        line_add = el.copy()
    
        for i in range(len(line_add)-1):
            line_add[i] = round(line_add[i] * value, 3)
        data_add.append(line_add)
        
    increase = [0.5,1,1.5,2,2.5,3,-0.5,-1,-1.5,-2,-2.5,-3]
    for value in increase:
        line_add = el.copy()
    
        for i in range(len(line_add)-1):
            line_add[i] = round(line_add[i] + value, 3)
        data_add.append(line_add)
'''

'\ndata_add = []\nfor el in data:\n    multiplicate = [2,2.5,3,0.5,0.2,0.1]\n    for value in multiplicate:\n        line_add = el.copy()\n    \n        for i in range(len(line_add)-1):\n            line_add[i] = round(line_add[i] * value, 3)\n        data_add.append(line_add)\n        \n    increase = [0.5,1,1.5,2,2.5,3,-0.5,-1,-1.5,-2,-2.5,-3]\n    for value in increase:\n        line_add = el.copy()\n    \n        for i in range(len(line_add)-1):\n            line_add[i] = round(line_add[i] + value, 3)\n        data_add.append(line_add)\n'

In [ ]:
'''
for el in data_add:
    data.append(el)
print(len(data))
'''

# y=0

In [ ]:
def change_one_value_in_sequence(line_ini, delta=0):
    line_add = line_ini.copy()
    line_add[i] = round(line_ini[i-1] + delta, 3)
    line_add[-1] = 0.
    return line_add
    

In [ ]:
data_add = []
for i in range(1, 6):
    for line in data:
        line_add = change_one_value_in_sequence(line, delta=0.002)
        data_add.append(line_add)
        
        line_add = change_one_value_in_sequence(line, delta=-0.002)
        data_add.append(line_add)
        
for line in data_add:
        data.append(line)
        
print(len(data))


# Write out

In [343]:
data_train = np.array(data)

file_csv = open('sources/train_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y'])
for i in range(data_train.shape[0]):
    filewriter.writerow(data_train[i,:])
    '''negative = list(map(lambda x: (-1)*x, data_train_1[i,:-1]))
    negative = np.flip(negative)
    add = data_train_1[i,:].copy()
    add[:-1] = negative
    filewriter.writerow(add)'''
'''for i in range(data_train_1.shape[0]):
    rand = data_train_1[i,:-1].copy()
    np.random.shuffle(rand)
    add = data_train_1[0,:].copy()
    add[:-1] = rand
    add[-1] = 0
    filewriter.writerow(add)
'''
file_csv.close()